Please first run:

%make install

in a terminal in this directory, and configure your runtime (VSCode, datahub, etc) to use the resultant virtual Python environment, or else uncomment the lines in the following cell

In [15]:
# pip install -r requirements.txt

In [16]:
# API key for Gov; substitute your own!
apikey = "rfaOy1m0PNZUGET08nKJvgbOrtpJaP86d8yt6RYh"

In [17]:
import sys
sys.path.append('./src')
from team_harriette_chick import getPrices
import pandas as pd

In [18]:
# from eep153_tools.sheets import read_sheets
import pandas as pd
import pickle

DRIs = None
with open('data/diet.pickle', 'rb') as fp:
    DRIs = pickle.load(fp)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

diet_max = DRIs['diet_maximums'].set_index('Nutrition')


In [19]:
import pandas as pd
import fooddatacentral as fdc
import warnings
import pandas as pd
import numpy as np
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

bulk_prices_w_lbs = pd.read_csv("data/bulk_prices.tsv", sep='\t')

apikey = "rfaOy1m0PNZUGET08nKJvgbOrtpJaP86d8yt6RYh"

D = {}
count = 0
food_names = bulk_prices_w_lbs.Material_Descr.tolist()
for food_name in food_names:
    try:
        fdc_id = bulk_prices_w_lbs.loc[bulk_prices_w_lbs.Material_Descr==food_name,:].FDC_ID
        int_fdcid = int(fdc_id[count])
        count+=1
        D[food_name] = fdc.nutrients(apikey,int_fdcid).Quantity
        print(f'count: {count} {food_name}')
    except AttributeError:
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food_name,int_fdcid))
    except ValueError:
        warnings.warn(f"No code for food {food_name}")
    except Exception as ex:
        warnings.warn(f"something seriously wrong for {food_name}  {ex}")

FoodNutrients = pd.DataFrame(D,dtype=float)

FoodNutrients = pd.DataFrame(D,dtype=float)

# all items are 1 pound
bulk_prices_w_lbs['FDC Quantity'] = bulk_prices_w_lbs[['Quantity','BaseUoM']].T.apply(lambda x : fdc.units(x['Quantity'],x['BaseUoM']))

bulk_prices_w_lbs['FDC Price'] = bulk_prices_w_lbs['Average_Price']/bulk_prices_w_lbs['FDC Quantity']

bulk_prices_w_lbs.dropna(how='any')

# To use minimum price observed
Prices = bulk_prices_w_lbs.groupby('Material_Descr',sort=False)['FDC Price'].min()

# our target group
group = 'F 9-13'
# tolerance
tol = 1e-6

result = getPrices.solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())



count: 1 CHEESE CHED WHT SHRED BAG-6/5 LB
count: 2 CHEESE PROCESS YEL SLC LVS-6/5 LB
count: 3 CHEESE MOZ LM PART SKM SHRD FRZ BOX-30LB
count: 4 CHEESE MOZ LM PART SKIM FRZ LVS-8/6 LB
count: 5 EGGS WHOLE FRZ CTN-6/5 LB
count: 6 EGGS WHOLE LIQ BULK -TANK
count: 7 CHICKEN LEGS CHILLED -BULK
count: 8 CHICKEN FAJITA STRIPS CTN-30 LB
count: 9 TURKEY BREAST DELI FRZ CTN-40 LB
count: 10 TURKEY BREAST SMKD DELI FRZ CTN-40 LB
count: 11 TURKEY ROASTS FRZ CTN-32-48 LB
count: 12 TURKEY HAMS SMKD FRZ CTN-40 LB
count: 13 BEEF CAN-24/24 OZ
count: 14 BEEF CRUMBLES W/SPP PKG-4/10 LB
count: 15 PORK CAN-24/24 OZ
count: 16 BEEF COARSE GROUND FRZ CTN-60 LB
count: 17 BEEF FINE GROUND FRZ CTN-40 LB
count: 18 BEEF PATTY LEAN FRZ CTN-40 LB
count: 19 PORK ROAST LEG FRZ CTN-32-40 LB
count: 20 PORK HAM WATERAD SLC FRZ PKG-8/5 LB
count: 21 PORK PICNIC BNLS FRZ CTN-60 LB
count: 22 CATFISH STRIPS BRD OVN RDY PKG-4/10 LB
count: 23 APPLE SLICES CAN-6/10
count: 24 MIXED FRUIT EX LT CAN-6/10
count: 25 APRICOTS DICED PEEL

count: 66 BEANS GREAT NORTHERN CAN-6/10
count: 67 BEANS PINTO DRY PKG-12/2 LB
count: 68 PEANUT BUTTER SMOOTH JAR-6/5 LB
count: 69 FLOUR ALL PURP ENRCH BLCH BAG-8/5 LB
count: 70 FLOUR WHOLE WHEAT BAG-50 LB
count: 71 FLOUR BAKER HARD WHT BLCH-BULK
count: 72 FLOUR BAKER HARD WHT UNBLCH-BULK
count: 73 PASTA SPAGHETTI CTN-20 LB
count: 74 OIL VEGETABLE BTL-6/1 GAL
count: 75 OATS ROLLED TUBE-12/42 OZ
count: 76 RICE US#1 LONG GRAIN PARBOILED BAG-25 LB
count: 77 RICE BRN US#1 LONG PARBOILED PKG-24/2 LB
count: 78 APPLES RED DELICIOUS FRESH CTN-40 LB
count: 79 MILK 1% MILKFAT UHT 2640 BOX-27/8 FL OZ
count: 80 CHICKEN BONED CAN-12/50 OZ
count: 81 FLOUR BREAD-BULK
count: 82 SUNFLOWER SEED BUTTER 6-5#'S
count: 83 RICE BRN US#1 LONG PARBOILED BAG-25 LB
count: 84 K APPLESAUCE CAN-6/10
count: 85 K PEACHES CLING CAN-6/10
count: 86 K CORN WHOLE KERNEL(LIQ) CAN-6/10
count: 87 K BEANS GREAT NORTHERN DRY BAG-25 LB
count: 88 K SUNFLOWER SEED BUTTER 6-5#'S
count: 89 CHICKEN OVEN ROASTED FRZ 8 PC CTN-30 LB
cou

/Users/christophernicholas/src/EEP153/EEP153_Harriette_Chick_Project2/venv/lib/python3.12/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


### Look up nutritional information for foods



## Effects of Price Changes on Subsistence Diet Cost



As prices change, we should expect the minimum cost diet to also
  change.  The code below creates a graph which changes prices away
  from the \`base&rsquo; case one food at a time, and plots changes in total
  diet cost.



In [20]:
import cufflinks as cf
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

cost0 = getPrices.solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = getPrices.solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        cost[Prices.index[i]] = np.log(result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T
Price_response.iplot(xTitle='change in log price',yTitle='change in log cost')

/var/folders/9r/yl209mt550g4h4t5v79dcyw80000gr/T/ipykernel_15307/2131278476.py:13: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

/var/folders/9r/yl209mt550g4h4t5v79dcyw80000gr/T/ipykernel_15307/2131278476.py:13: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

/var/folders/9r/yl209mt550g4h4t5v79dcyw80000gr/T/ipykernel_15307/2131278476.py:13: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

/var/folders/9r/yl209mt550g4h4t5v7

KeyboardInterrupt: 

## Effects of Price Changes on Subsistence Diet Composition



The code below creates a graph which changes prices just for *one* food,
  and traces out the effects of this change on all the foods consumed.



In [21]:
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'EGGS WHOLE LIQ BULK -TANK'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = getPrices.solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = getPrices.solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

/Users/christophernicholas/src/EEP153/EEP153_Harriette_Chick_Project2/venv/lib/python3.12/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



## Effects of Price Changes on Subsistence Diet Nutrition



The code below creates a graph which uses the food price changes
  described above, but maps into nutrients.



In [ ]:
# Matrix product maps quantities of food into quantities of nutrients
NutrientResponse = Diet_response@FoodNutrients.T

# Drop columns of missing nutrients
NutrientResponse = NutrientResponse.loc[:,NutrientResponse.count()>0]
NutrientResponse.iplot(xTitle='%s Price' % ReferenceGood,yTitle='Nutrients')

## Adding Constraint on Total Weight



At least at some prices the minimum cost subistence diet involves
eating unreasonable amounts of food (e.g., 10 kilograms of cabbage per
day).  We can easily add an additional constraint of the form
$$
     \sum x_i \leq \text{max weight}
$$
to our linear programming problem since it&rsquo;s just another linear
inequality, and this may give us more realistic results.



### Price Changes and Subsistence Diet Composition with Weight Constraint



Re-do our analysis of changing prices, but with a constraint that
  total weight of diet must be less that 12 hectograms (1.2 kg).



In [22]:
# import cufflinks as cf
# cf.go_offline()

ReferenceGood = 'EGGS WHOLE LIQ BULK -TANK'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = getPrices.solve_subsistence_problem(FoodNutrients,Prices,
                                  diet_min[group],diet_max[group],max_weight=12,tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = getPrices.solve_subsistence_problem(FoodNutrients,my_p,
                                       diet_min[group],diet_max[group],max_weight=12,tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

/Users/christophernicholas/src/EEP153/EEP153_Harriette_Chick_Project2/venv/lib/python3.12/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

